<a href="https://colab.research.google.com/github/anishgupta02/PDF_Querying_using_Langchain/blob/main/PDF_Query_using_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [31]:
import os
os.environ["OPENAI_API_KEY"] = "GIVE_YOUR_KEY"

In [32]:
pdfreader = PdfReader('/content/MachineTranslationwithAttention.pdf') #give the path of your data

In [5]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [6]:
raw_text

'See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/355917108\nNeu ral Machine T ranslation with Attention\nTechnic al R eport  · August 2021\nDOI: 10.13140/RG.2.2.29381.37607/1\nCITATIONS\n0READS\n2,017\n2 author s:\nMohammad W asil Saleem\nUniv ersität P otsdam\n3 PUBLICA TIONS \xa0\xa0\xa00 CITATIONS \xa0\xa0\xa0\nSEE PROFILE\nSandeep Upr ety\nUniv ersität P otsdam\n1 PUBLICA TION \xa0\xa0\xa00 CITATIONS \xa0\xa0\xa0\nSEE PROFILE\nAll c ontent f ollo wing this p age was uplo aded b y Sandeep Upr ety on 05 No vember 2021.\nThe user has r equest ed enhanc ement of the do wnlo aded file.Neural Machine Translation with Attention\nMohammad Wasil Saleem\nMatrikel-Nr.: 805779\nUniversit ¨at Potsdam\nsaleem1@uni-potsdam.deSandeep Uprety\nMatrikel-Nr. 804982\nUniversit ¨at Potsdam\nuprety@uni-potsdam.de\nAbstract\nIn recent years, the success achieved\nthrough neural machine translation has\nmade it mainstream in machine tran

In [21]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 300,
    chunk_overlap  = 50,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [22]:
len(texts)

135

In [23]:
texts

['See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/355917108\nNeu ral Machine T ranslation with Attention\nTechnic al R eport  · August 2021\nDOI: 10.13140/RG.2.2.29381.37607/1\nCITATIONS\n0READS\n2,017\n2 author s:\nMohammad W asil Saleem',
 '0READS\n2,017\n2 author s:\nMohammad W asil Saleem\nUniv ersität P otsdam\n3 PUBLICA TIONS \xa0\xa0\xa00 CITATIONS \xa0\xa0\xa0\nSEE PROFILE\nSandeep Upr ety\nUniv ersität P otsdam\n1 PUBLICA TION \xa0\xa0\xa00 CITATIONS \xa0\xa0\xa0\nSEE PROFILE\nAll c ontent f ollo wing this p age was uplo aded b y Sandeep Upr ety on 05 No vember 2021.',
 'The user has r equest ed enhanc ement of the do wnlo aded file.Neural Machine Translation with Attention\nMohammad Wasil Saleem\nMatrikel-Nr.: 805779\nUniversit ¨at Potsdam\nsaleem1@uni-potsdam.deSandeep Uprety\nMatrikel-Nr. 804982\nUniversit ¨at Potsdam\nuprety@uni-potsdam.de\nAbstract',
 'uprety@uni-potsdam.de\nAbstract\nIn recent years, t

In [24]:
embeddings = OpenAIEmbeddings()

In [25]:
document_search = FAISS.from_texts(texts, embeddings)

In [26]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [27]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [28]:
query = "Why attention is used?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Attention is used to help memorize information for longer sentences and to address the issue of the encoder not being able to memorize words at the beginning of sentences, leading to poor translation.'

In [29]:
query = "Summarize the data and give the results and discussions breifly"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

" The paper discusses the results of a machine learning model trained on sentences of a maximum length of 50. The model was tested on multiple languages, including German and French, and achieved a BLEU score of 25.37. The authors note that there is room for improvement, as hardware limitations may have affected the results. They also mention that their goal was to replicate the results of a previous paper by Bahdanau et al. (2014), and their achieved score was close to the original paper's."

In [30]:
query = "what are the limitations and disadvatages based on results and discusions"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

'The limitations and disadvantages mentioned in the context include hardware limitations that prevented achieving better results, the use of fixed length vectors in a simple encoder decoder model that underperformed with long sentences, and the time and maintenance constraints of using a university server and Google Colab. These limitations and constraints may have affected the performance and overall results of the proposed approach by Bahdanau et al. (2014). Additionally, the qualitative results of the model also suggest potential limitations and areas for improvement, such as the soft alignment between translated and input sentences and the weights assigned to each cell in the matrix.'